<a href="https://colab.research.google.com/github/nyamilanzi/Web-scraping-pj/blob/main/copyofjobsearchmw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get update
!apt-get install -y wget
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -y -f

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time


chrome_driver_path = '/content/drive/My Drive/chromedriver.exe'

driver = webdriver.Chrome()

#driver = webdriver.Chrome()
driver.get("https://jobsearchmalawi.com/")

SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x5893e29e9df3 <unknown>
#1 0x5893e26d84e7 <unknown>
#2 0x5893e270ca66 <unknown>
#3 0x5893e27089c0 <unknown>
#4 0x5893e27517f0 <unknown>
#5 0x5893e27451f3 <unknown>
#6 0x5893e271628a <unknown>
#7 0x5893e2716c5e <unknown>
#8 0x5893e29ae11b <unknown>
#9 0x5893e29b206b <unknown>
#10 0x5893e299a231 <unknown>
#11 0x5893e29b2bd2 <unknown>
#12 0x5893e297f0ef <unknown>
#13 0x5893e29d8f48 <unknown>
#14 0x5893e29d9120 <unknown>
#15 0x5893e29e8f44 <unknown>
#16 0x7df95b891ac3 <unknown>


In [ ]:
def scrape_job_description(url):
    driver.get(url)
    try:
        job_description_elem = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "job_description"))
        )
        job_description = job_description_elem.text.strip()
    except:
        job_description = None
    return job_description

def scrape_listings(page_source, processed_links):
    soup = BeautifulSoup(page_source, 'html.parser')
    job_listings = soup.find_all('li', class_='job_listing')
    listings_data = []
    for listing in job_listings:
        job_position_elem = listing.find('h3')
        job_position = job_position_elem.text.strip() if job_position_elem else None
        if job_position == "How to Search For a Job Advert" or job_position == "Copycats of Our Site  (Dont Support Laziness)":
            continue

        company_elem = listing.find('div', class_='company')
        company = company_elem.strong.text.strip() if company_elem and company_elem.strong else None
        if company is None:
            continue

        location_elem = listing.find('div', class_='location')
        location = location_elem.text.strip() if location_elem else None
        if location is None:
            continue

        job_type_elem = listing.find('li', class_='job-type')
        job_type = job_type_elem.text.strip() if job_type_elem else None
        if job_type is None:
            continue

        posting_date_elem = listing.find('li', class_='date')
        posting_date = posting_date_elem.time['datetime'] if posting_date_elem else None
        if posting_date is None:
            continue

        job_link_elem = listing.find('a', href=True)
        job_link = job_link_elem['href'] if job_link_elem else None

        if job_link not in processed_links:
            listing_data = {
                'Position': job_position,
                'Company': company,
                'Location': location,
                'Job Type': job_type,
                'Posting Date': posting_date,
                'Job Link': job_link
            }
            listings_data.append(listing_data)
            processed_links.add(job_link)
    return listings_data

new_listings = []
processed_links = set()

while len(new_listings) < 650000:
    try:
        load_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "load_more_jobs"))
        )
        load_more_button.click()
        time.sleep(2)
        page_source = driver.page_source
        scraped_listings = scrape_listings(page_source, processed_links)
        if not scraped_listings:
            print("No more listings found.")
            break
        new_listings.extend(scraped_listings)
        print("Processed links:", processed_links)
        print("Total listings:", len(new_listings))
    except Exception as e:
        print("An error occurred:", e)
        break

if new_listings:
    df = pd.DataFrame(new_listings[:650000])
    df.to_csv('jobsearch_listings.csv', index=False)

    descriptions = []
    for job_link in df['Job Link']:
        description = scrape_job_description(job_link)
        descriptions.append(description)

    df['Job Description'] = descriptions

    df.to_csv('jobsearch_listings_with_descriptions.csv', index=False)
    print("Job descriptions scraped successfully.")
else:
    print("No new listings found.")

driver.quit()